Import quantfinanceHEC library.

In [7]:
import pandas as pd
import quantfinanceHEC as qf

Use data fetching function to get aapl and spy close price data between the defined start and end dates.

In [21]:
start_date = "2020-01-01"
end_date = "2021-01-01"

aapl = qf.data_fetching.f_fetch_stock_data(symbol="aapl", start_date=start_date, end_date=end_date, interval="daily")["4. close"]
spy = qf.data_fetching.f_fetch_stock_data(symbol="spy", start_date=start_date, end_date=end_date, interval="daily")["4. close"]
prices = pd.concat([aapl, spy], axis = 1)
prices.columns = ["aapl", "spy"]

We now have a pandas dataframe containing our close prices.

In [29]:
prices = prices.iloc[::-1] # reverse order of index
prices

,aapl,spy
date,,
2020-01-02,300.35,324.87
2020-01-03,297.43,322.41
2020-01-06,299.80,323.64
2020-01-07,298.39,322.73
2020-01-08,303.19,324.45
...,...,...
2020-12-24,131.97,369.00
2020-12-28,136.69,372.17
2020-12-29,134.87,371.46


Lets calculate the returns.

In [34]:
returns = prices.pct_change()
returns.dropna()


,aapl,spy
date,,
2020-01-03,-0.009722,-0.007572
2020-01-06,0.007968,0.003815
2020-01-07,-0.004703,-0.002812
2020-01-08,0.016086,0.005330
2020-01-09,0.021241,0.006781
...,...,...
2020-12-24,0.007712,0.003890
2020-12-28,0.035766,0.008591
2020-12-29,-0.013315,-0.001908
